Detections Database and API
=======================

This notebook demonstrates how to access the HDF5 container for the HETDEX line detections database. This database is a catalog of line emission detections and their associated 1D, aperture summed, psf-weighted spectra. There are three tables contained within this HDF5 file: 

1. Detections - this is the main database of line detection sources. It provides the position and central wavelength of each detection and corresponding line fluxes. A source detection corresponds to an emission line so it is possible to have multiple line detections at different wavelengths for a single source. There can also be multiple observations of the same line if it has been observed in multiple shots or if it is associated with a large source. 


2. Fibers - for each source detection, this table lists information about each fiber used to extract the flux measurment and weighted spectrum. This allows a user to return to the processed data products (ie. the shot HDF5 files) to investigate the source further. 


3. Spectra - for each source, this table contains arrays of wavelength and 1D flux-weighted aperture summed spectral data and corresponding errors. Non-calibrated spectra is also provided in counts


4. Elixer - a direct row matching of Elixer classification info for each row in the Detections table. The hetdex gmag value is also found in this table. For full Elixer data including catalog and aperture P(LAE)/P(OII) values please see the detects/elixer.h5 file for a specific release. These are the ones currently accessible in the API and the curated catalogs:

    "plae_sdss_g" = P(LAE)/P(OII) from g-mag with HETDEX 1 D spectrum limits in plae_sdss_g_max/min
    
    "combined_plae"  = ELiXer weighted combination of the multiple PLAE/POII with error in combine_plae_err
    
    "plae_classification" = ELiXer probability that the line is LyA (0.0 - 1.0) [WARNING! Very experimental!!] 



In [ ]:
%matplotlib inline

import numpy as np
import tables as tb
import matplotlib.pyplot as plt

from astropy.table import Table, Column, join
from astropy.coordinates import SkyCoord
import astropy.units as u

from hetdex_api.config import HDRconfig
from hetdex_api.detections import Detections
from hetdex_api.elixer_widget_cls import ElixerWidget

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

### Use the latest curated catalog: 

In [ ]:
detects = Detections( curated_version='2.1.2')

In [ ]:
# this is a suggested query to find LAEs:
sel_lae = (detects.sn > 5.5) * (detects.plae_classification > 0.75) * (np.logical_not(detects.gmag<20))

In [ ]:
#spectra can be accessed using get_spectrum()
spec = detects.get_spectrum(2100191119)

In [ ]:
spec

### Explore Using ElixerWidget:

In [ ]:
elix_widget = ElixerWidget(detectlist = detects.detectid[sel_lae])

### Initiate the API and Access the full database

When you call `Detections()` you intiate the Detections Class object which takes columns from the Detections Table in the HDF5 file and adds them as array attributes to the Detections class object. It also converts ra/dec into astropy skycoords in the `coords` attribute, calculates an approximate gband magnitude using the 1D spectra and adds elixer probabilities for each detection. If you append the call with `refine()` then a number of downselections are applied to the database to return a more robust list of line emitters. `refine()` removes spurious detections found in bad amps or at the edges of the CCD or in shots that are not deemed appropriate for HETDEX analysis. It can also remove all bright objects above a specific gband magnitude if desired (default to None if no option is given).

In [ ]:
# To access the latest HDRX.X lines database (ie. the full H5 file):

detects = Detections()

# to remove the latest bad amps and pixels (this isn't needed if you are using a curated catalog)

# detects = Detections(survey='hdr2.1', catalog_type='lines').refine()

# or if you want to open the continuum source catalog:
# detects = Detections(survey='hdr2.1', catalog_type='continuum')

### Note if you do not want to load the whole table, but just access spectra for a specific detectid:

In [ ]:
det_object = Detections('hdr2.1', loadtable=False)

In [ ]:
spec = det_object.get_spectrum(2100191119)

In [ ]:
spec

Here are a list of attributes built into the Detections class:

In [ ]:
detects.__dict__.keys()

If you prefer working in astropy tables, you can grab it this way:

In [ ]:
detect_table = detects.return_astropy_table()

In [ ]:
detect_table

## How we made the subset catalog for the team:

In [ ]:
sel_field = (detects.field == 'cosmos') | (detects.field == 'dex-fall') | (detects.field == 'dex-spring') | (detects.field == 'egs') | (detects.field == 'goods-n')
sel_chi2 = detects.chi2 < 1.2
sel_wave = ( detects.wave >= 3510 ) * (detects.wave <= 5490)
sel_lw = (detects.linewidth <= 6)
sel_cont = detects.continuum > -3
sel_sn = detects.sn >= 4.8
sel_chi2fib = (detects.chi2fib < 4.5)

sel_cat = sel_field * sel_chi2 * sel_wave * sel_lw * sel_cont * sel_sn * sel_chi2fib

det_table = detects.return_astropy_table()

In [ ]:
team_table = detect_table[sel_cat]

## Querying by sky coordinates

Upon initialization of the Detections Class, sky coordinates are converted to an Astropy sky coordinates array to allow for easy querying:

In [ ]:
detects.coords

To query a region of the sky, you can use the Detections function `query_by_coords` which takes an astropy coords objects as an argument as well as a radius represented by an astropy quantity. It returns a boolean mask to index the Detections class object.

In [ ]:
obj_coords = SkyCoord(199.35704 * u.deg, 51.06718 * u.deg, frame='icrs')

In [ ]:
maskregion = detects.query_by_coords(obj_coords, 10. * u.arcsec)

The Detections class allows slicing so that a boolean mask applied to the class will slice each array attribute accordingly:

In [ ]:
detects_in_region = detects[maskregion]
print(np.size(detects_in_region.detectid))

## Find a direct line match

If you want to find an exact line match you can use the function `find_match()`

In [ ]:
obj_coords = SkyCoord(199.35704 * u.deg, 51.06718 * u.deg, frame='icrs')

In [ ]:
wave_obj = 3836.

In [ ]:
idx = detects.find_match(obj_coords, wave=wave_obj, radius=5.*u.arcsec, dwave=5 )

In [ ]:
detects.detectid[idx]

In [ ]:
detect_table[idx]

## Check out matched sources in the ElixerWidget

For this example, we have found 12 detections in this region, we can examine these via the ELiXer reports using the `ElixerWidget()` class from `hetdex_api.elixer_widget_cls.py`. To do so we need to save the detectid list to examine in the widget.

In [ ]:
#np.savetxt('detects_obj.txt', detects_in_region.detectid)

You can the run the elixer_widget to scan through the ELiXer reports for this object. Use the "Next DetectID" button to scan the list. The "DetectID" text widget will give access to all reports interactively and scans in increasing single digit increments, but the green Next DetectID button will go in order of the ingest list from 'detects_obj.txt'.

In [ ]:
elix_widget = ElixerWidget(detectlist = detects_in_region.detectid)
#elix_widget = ElixerWidget(detectfile='detects_obj.txt')

For more information on using the Elixer Widgets GUI go to Notebook 12. We will discuss team classification efforts there. But for quick investigation its helpful to pull the GUI up to just scan through a detection list.

## Accessing 1D Spectra

Spectra in counts and flux-calibrated units are stored in the Spectra Table of the Detection HDF5 file, it can be accessed directly through the Detections class object which stores the detect HDF5 as an attribute:

In [ ]:
print(detects.hdfile)

In [ ]:
spectra = detects.hdfile.root.Spectra

This is a very large table so its not advised to read it in all at once. The columns are:

In [ ]:
spectra.cols

Flux calibrated, psf-weighted 1D spectra can be retrieved via the API for a single detectid through the function `get_spectrum`:

In [ ]:
detectid_nice_lae = 2100744791
spec_table = detects.get_spectrum(detectid_nice_lae) 

In [ ]:
detects.plot_spectrum(detectid_nice_lae)

or if we want to zoom in on the emission line:

In [ ]:
cw = detects.wave[detects.detectid == detectid_nice_lae]
detects.plot_spectrum(detectid_nice_lae, xlim=(cw-50, cw+50))

You can also save the spectrum to a text file. It is automatically saved as spec_##detectid##.dat, but you can also use the argument `outfile`

In [ ]:
detects.save_spectrum(detectid_nice_lae)
# or
# detects.save_spectrum(detectid_nice_lae, outfile='tmp.txt')

## Getting Fiber information for a detection

You can find a list of all fibers used in the measurement in the Fibers table. The Fibers table and its associated columns can be accessed similar to the Spectra table by searching for a match in the the detectid column. 

In [ ]:
fibers = detects.hdfile.root.Fibers
fibers.cols

Access the fiber table for the above source:

In [ ]:
fiber_table = fibers.read_where("detectid == detectid_nice_lae") 

In [ ]:
Table(fiber_table)

When you are done with the HDF5 file, close it. The data that you extracted into tables and arrays will remain.

In [ ]:
detects.hdfile.close()

## Accessing the ELiXer Classifications

In [ ]:
config = HDRconfig(survey='hdr2.1')
file_elix = tb.open_file(config.elixerh5)

In [ ]:
file_elix.root.Detections

Note: these are also appended to the Detections() class object. Each column in the above table can be accessed as an attribute of the Detections() class object. For example, the probability of LAE to OII measured from the HETDEX continuum is:

In [ ]:
#detects.plae_poii_hetdex

or the nearest neighbour magnitude in an ancillary photometric catalog is:

In [ ]:
#detects.mag_match

and this comes from the filter:

In [ ]:
#detects.cat_filter